In [13]:
import datasets
import torch.nn as nn
import data_process as dp
import torchtext

In [14]:
moves_df = datasets.load_sql_to_df("SELECT * FROM english_annotated_moves", "./chess.db")
moves_df.head(5)

,index,position,move,comment,halfmove_number,game_id,sentiment,auto_sentiment
0,0,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,e2e4,This is my first gameknot game against someone...,1,44,2,0
1,1,rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP1PPP/RNBQKBN...,b1c3,"I've been playing the Vienna Gambit as white, ...",3,44,-1,0
2,2,rnbqkbnr/pppp1ppp/8/4p3/4P3/2N5/PPPP1PPP/R1BQK...,f8c5,Minor disappointment.,4,44,-1,0
3,3,r1bqk2r/pppp1ppp/2n2n2/2b1p3/2B1P3/2NP4/PPP2PP...,f2f4,"My idea here is to expand on the kingside, dri...",9,44,-1,0
4,4,r1bqk2r/ppp2ppp/3p1n2/n1b1pP2/2B1P3/2NP4/PPP3P...,d1f3,"Maybe this isn't the greatest plan, since with...",13,44,0,0


In [15]:
glove_embbedings = torchtext.vocab.GloVe(name="6B", dim=50)

In [18]:
moves_df = dp.prepare_data_for_sentiment_analysis_training(moves_df, glove_embbedings.stoi)
moves_df

,comment,sentiment
0,"[maybe, this, be, not, the, great, plan, since...",0
1,"[?, too, slow, maybe, .]",0
2,"[?, this, allow, a, combination, by, white, .,...",0
3,"[!, !, brilliant, move, it, give, black, the, ...",1
4,"[!, !, this, move, be, really, fantastic, ., m...",1
...,...,...
19864,"[in, hindsight, be, need, to, get, my, king, t...",0
19865,"[keep, black, alive]",0
19866,"[time, remain, minute, .]",0
19867,"[black, be, win, .]",1


In [33]:
data = list(zip(moves_df.comment, moves_df.sentiment))
datasets.add_padding_vector_to_embeddings(glove_embbedings)
dataset = datasets.PretrainedEmbeddingsIndicesDataset(data, glove_embbedings)

In [34]:
x,y = dataset[5000]

In [36]:
print([glove_embbedings.itos[t.item()] for t in x])
print(x)
print(y)

['white', 'go', 'for', 'the', 'jugular', 'and', 'immediately', 'storm', 'my', 'kingside', '.']
tensor([   299,    243,     11,      1,  58824,      6,   1041,   1837,    193,
        130538,      3])
tensor(1)


In [12]:
import spacy
nlp = spacy.load("en_core_web_sm", disable = ['parser','ner'])

2023-02-14 12:08:26.140029: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-14 12:08:27.306368: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-14 12:08:27.306431: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-02-14 12:08:27.306438: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [16]:
docs = [t for t in nlp.pipe(glove_embbedings.itos, n_process=16, batch_size=128)]

In [17]:
lemmas = [dp.doc_to_lemma_tokens(tokens, glove_embbedings.stoi) for tokens in docs]

['`', '`']
['u.s', '.']
["'", 've']
["'", 'm']
['&', 'amp', ';']
['u.n', '.']
['inc', '.']
['st', '.']
['no', '.']
['corp', '.']
['can', 'not']
['dr', '.']
['sept', '.']
['sen', '.']
['mr', '.']
['oct', '.']
['dec', '.']
['jr', '.']
['jan', '.']
['nov', '.']
['ms', '.']
['aug', '.']
['gen', '.']
['rep', '.']
['feb', '.']
['6', '-', '3']
['6', '-', '4']
['ltd', '.']
['mrs', '.']
['6', '-', '2']
['gov', '.']
['d.c', '.']
['1', '-', '0']
['_', '_', '_']
['calif', '.']
['2', '-', '1']
['2', '-', '0']
['n.y', '.']
['6', '-', '1']
['etc', '.']
['7', '-', '6']
['lt', '.']
['10', '-', 'year']
['3', '-', '0']
['fla', '.']
['rev', '.']
['7', '-', '5']
['col', '.']
['1', '-', '1']
['3', '-', '1']
['30', '-', 'year']
['3', '-', '2']
['l.a', '.']
['_', '_']
['va', '.']
['maj', '.']
['6', '-', '0']
['u.k', '.']
['20', '-', 'year']
['0', '-', '0']
['n.j', '.']
['2', '-', '2']
['4', '-', '1']
[',', '000']
['4', '-', '6']
['capt', '.']
['15', '-', 'year']
['n.c', '.']
['well', '-', 'know']
['25', '-', 